In [52]:
import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt
import matplotlib
from datetime import datetime
import re
import sklearn
import seaborn as sns
import collections
from scipy.stats import norm
from sklearn.preprocessing import LabelEncoder
pd.set_option('display.max_columns', None)

In [53]:
df = pd.read_excel('../Data/OriginalData/TMDB_dataset.xlsx')
df = df.dropna()
df.reset_index(drop=True, inplace=True)

**<font size="5">Popularity Metrics</font>**

**Row-Wise Function to Compute Average Popularity for a Movie Crew/Cast**

In [54]:
def avg_popularity(row):
    score = 0
    remove_duplicates = []
    if isinstance(row, float):
        return np.nan
    else:
        for person in row.split(','):
            if person not in remove_duplicates:
                remove_duplicates.append(person)
        for m in remove_duplicates:
            if len(m.lstrip().split(':')) > 1:
                score += float(m.lstrip().split(':')[-1])
        return score/len(remove_duplicates)

**Function Call**

In [55]:
df['cast_average'] = df['cast'].apply(lambda c: avg_popularity(c))
df['crew_average'] = df['crew'].apply(lambda c: avg_popularity(c))

**<font size="5">Encodings</font>**

**OneHot Encode the top 500 actors**  
Note: Cannot use built in function because we need to change the format

In [56]:
unique_actors = []
for list_of_actors in df['cast'].tolist():
    for act in list_of_actors.split(','):
        unique_actors.append(act.split(':')[0].lstrip())

In [57]:
common_act = [c[0] for c in collections.Counter(unique_actors).most_common(500)]
for act in common_act:
    df[act] = df['cast'].apply(lambda actor_list: 1 if act in actor_list.split(', ') else 0)
df = df.drop('cast', axis=1)

/var/folders/wf/gnllpg551yvc7g8z_j80tmgm0000gn/T/ipykernel_13430/1061978767.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  df[act] = df['cast'].apply(lambda actor_list: 1 if act in actor_list.split(', ') else 0)


**OneHot Encode the top 500 crew members**  
Note: Cannot use built in function because we need to change the format

In [58]:
unique_crew = []
for list_of_crew in df['crew'].tolist():
    for mem in list_of_crew.split(','):
        unique_crew.append(mem.split(':')[0].lstrip())

In [59]:
common_mems = [c[0] for c in collections.Counter(unique_crew).most_common(500)]
for mem in common_mems:
    df[mem] = df['crew'].apply(lambda crew_list: 1 if mem in crew_list.split(', ') else 0)
df = df.drop('crew', axis=1)

/var/folders/wf/gnllpg551yvc7g8z_j80tmgm0000gn/T/ipykernel_13430/4245124629.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  df[mem] = df['crew'].apply(lambda crew_list: 1 if mem in crew_list.split(', ') else 0)


**OneHot Encode the Genres**  
Note: Cannot use built in function because we need to change the format

In [60]:
unique_genres = []
for list_of_genres in df['genres'].tolist():
    for genre in list_of_genres.split(','):
        if genre.lstrip() not in unique_genres:
            unique_genres.append(genre.lstrip())

In [61]:
for genre in unique_genres:
    df[genre] = df['genres'].apply(lambda genre_list: 1 if genre in genre_list.split(', ') else 0)
df = df.drop('genres', axis=1)

**OneHot Encode the Production Companies (May not use this attribute because there are 9313 companies)**  
Note: Cannot use built in function because we need to change the format

In [62]:
unique_prod_comp = []
for list_of_comp in df['production_companies'].tolist():
    for comp in list_of_comp.split(','):
        unique_prod_comp.append((comp.lstrip()))

In [63]:
common_pc = [c[0] for c in collections.Counter(unique_prod_comp).most_common(50)]
for comp in common_pc:
    df[comp] = df['production_companies'].apply(lambda comp_list: 1 if comp in comp_list.split(', ') else 0)
df = df.drop('production_companies', axis=1)

**OneHot Encode the Original Langauge**  
Note: Also may leave out because there are 50 different languages

In [64]:
df = pd.get_dummies(df, columns=['original_language'])

**Bin Release Dates By Year**

In [65]:
# Convert to datetime
df['release_date'] = df['release_date'].apply(lambda d: datetime.strptime(d, '%Y-%m-%d'))
# Bin and Encode
le = LabelEncoder()
n_bins = 104 # Number of years from 1918 and 2020 (inclusive)
df['release_date'] = le.fit_transform(pd.cut(df['release_date'], n_bins, retbins=True)[0])

**Create New Columns for Day of Month and Year (worse than binning)**

In [66]:
# # Convert to datetime
# df['release_date'] = df['release_date'].apply(lambda d: datetime.strptime(d, '%Y-%m-%d'))
# df['day'] = df['release_date'].dt.day
# df['month'] = df['release_date'].dt.month
# df['year'] = df['release_date'].dt.year
# df = df.drop(['release_date'], axis=1)

**Log Transform Revenue and Budget**

In [67]:
df['log_revenue'] = np.log1p(df['revenue'])
df['log_budget'] = np.log1p(df['budget'])

**Save to New File**

In [68]:
df.to_excel('../Data/ProcessedData/TMDB_processed.xlsx', index=False)